## Обработка датасета для генератора

Для работы генератора требуется понимание с каких IP-адресов на какие компьютеры (IP-адреса), с каких портов и на какие порты отправляются сетевые пакеты с данными как при атаках, так и при обычных запросах. Это необходимо знать для того, чтобы сгенерировать пакеты с данными, которые будут приближены к данным, на которых обучен градиентный бустинг и нейронная сеть для детектирования атак. 
В этом файле описано, как вычленить из нашего IP-адреса машин, с которых отправляются данные, сгруппировать их по признаку боты это или обычные пользовательские комьютеры. И для каждой такой машины агрегировать список ip-адресов на которые она отправляла запросы, список портов с которых отправлялись сетевые пакеты, список портов на которые отправлялись пакеты, среднее время отправки пакетов данных, средняя длина пакетов, среднее количество пакетов во время одной сессии и т.п. Эти данные позволят нам генерировать похожие пакеты для найденных ip-адресов комьютеров.

In [1]:
import numpy as np
import pandas as pd

Для примера был взят файл с https://www.kaggle.com/devendra416/ddos-datasets. Был взят датасет из папки ddos_balanced. Т.к. размер файла с данными очень большой (более 6 ГБ) необходимо применить специальные механизмы для загрузки данных. Для этого:

1. Файл загружался блоками (chunks) по 400 тыс. строк в каждом блоке (значение переменной size).
2. При загрузке данных они сразу сжимались, путем группировки данных по ip-адресу машины источника данных  (Поле Src IP).
3. Вычислялись средние числовые показатели, а так же группировались во множества (set) нечисловые(категориальные) параметры (порт источника, порт назначения, ip-адрес назначения и др.). 

Данные группируются во множество, остаются только уникальные значения этих параметров в рамках данного блока (chunk) данных из большого файла, т.е. для одного ip-адреса в поле Dst IP будут храниться только уникальные ip-адреса, для Protocol - только уникальные
номера протоколов по которым отправлял данные этот комьютер и т.д. Это значительно сокращает объем получаемых данных.

In [10]:
size = 4 * 10**4 # Размер блока данных
readerCSV = pd.read_csv("/Users/evgenii/DDoS Dataset/final_dataset.csv",sep=",",chunksize=size)

# каждый блок таких данных будет храниться в этом списке
ddos_result = []
i = 0
number_of_objects = 12794627
iter_count = number_of_objects/size 

for data in readerCSV:
     # заполняем пропуски нулями - полезно для расчета средних значений непрерывных переменных
    data = data.fillna(0)
    # группируем данные по ip-адресу источника пакетов и вычисляем средние показатели по всем числовы параметром
    # из общего числа параметров были выбраны только некоторые, которые играют важную роль при формировании пакетов
    # и помещаются в памяти при загрузке. Полный перечень данных в памяти не поместился
    cnt_data = data.groupby(['Src IP']).mean()[['Flow Duration', 'Tot Fwd Pkts', 'Tot Bwd Pkts', 'TotLen Fwd Pkts', 
                                               'TotLen Bwd Pkts', 'Fwd IAT Mean' , 'Bwd IAT Mean' , 'Fwd PSH Flags', 
                                               'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'Pkt Len Mean', 
                                               'SYN Flag Cnt', 'RST Flag Cnt', 'PSH Flag Cnt', 'ACK Flag Cnt' , 
                                               'URG Flag Cnt', 'CWE Flag Count', 'ECE Flag Cnt', 'Idle Mean', 'Active Mean']]
    # группируем данные по ip-адресу источника пакетов и агрегируем категориальные параметры во множества
    dst_ip_data = data.groupby(['Src IP']).agg({'Dst IP': set})
    protocol_data = data.groupby(['Src IP']).agg({'Protocol': set})
    dst_port_data = data.groupby(['Src IP']).agg({'Dst Port': set})
    src_ports_data = data.groupby(['Src IP']).agg({'Src Port': set})
    labels_data = data.groupby(['Src IP']).agg({'Label': set})
      # объединяем получившиеся отдельные наборы данных единую таблицу
    ddos_data = src_ports_data.join(dst_ip_data).join(dst_port_data).join(protocol_data).join(labels_data).join(cnt_data)
    
    # добавляем таблицу данных для данного блока из файла в общий список блоков
    ddos_result.append(ddos_data)
    
    i += 1
    percent = i / iter_count * 100
    print("Чтение данных: " + "{0:.2f}".format(percent if percent < 100 else 100) + "%", end="\r")
print()
print('finished')
readerCSV = None
data = None

Чтение данных: 100.00%
finished


In [11]:
# соединяем отдельные блоки полученных данных из файла (предварительно уже преобразованных по правилам, 
# которые описаны выше) в единую таблицу
intermediate_res = pd.concat(ddos_result,sort=False)
ddos_result = []
intermediate_res

,Src Port,Dst IP,Dst Port,Protocol,Label,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,...,Pkt Len Mean,SYN Flag Cnt,RST Flag Cnt,PSH Flag Cnt,ACK Flag Cnt,URG Flag Cnt,CWE Flag Count,ECE Flag Cnt,Idle Mean,Active Mean
Src IP,,,,,,,,,,,,,,,,,,,,,
192.168.1.101,"{4100, 2055, 2057, 2061, 2068, 2069, 2076, 208...","{97.74.104.201, 74.55.1.4, 97.74.144.108}","{80, 443}",{6},{ddos},1.563599e+07,6.884336,8.625909,186.352280,7788.161269,...,207.223648,0.458030,0.0,0.002644,0.541970,0.0,0.0,0.0,6.644611e+06,8.336376e+04
192.168.1.102,"{3076, 3077, 3079, 3082, 3083, 3088, 3090, 309...","{69.84.133.138, 74.55.1.4, 97.74.104.201}",{80},{6},{ddos},1.664876e+07,84.918768,159.750700,293.344538,216019.442577,...,385.028472,0.521008,0.0,0.005602,0.478992,0.0,0.0,0.0,1.003268e+06,3.734518e+05
192.168.1.103,"{2049, 2563, 2052, 2053, 2054, 3075, 3258, 358...","{69.192.24.88, 69.84.133.138, 208.113.162.153,...",{80},{6},{ddos},2.002657e+07,38.111709,62.118439,206.213997,83564.204576,...,318.998784,0.410498,0.0,0.008075,0.589502,0.0,0.0,0.0,1.711862e+06,5.116172e+05
192.168.1.104,"{21574, 18449, 18450, 18451, 20523, 20524, 205...","{69.192.24.88, 97.74.144.108, 97.74.104.201, 6...","{80, 443}",{6},{ddos},6.761965e+06,5.443097,9.551819,65.679312,10077.454830,...,179.230691,0.474775,0.0,0.000000,0.525225,0.0,0.0,0.0,3.850293e+05,1.631446e+04
192.168.1.105,"{17408, 18560, 17412, 17418, 18582, 18585, 184...",{97.74.104.201},"{80, 443}",{6},{ddos},5.341630e+07,71.464286,124.642857,350.821429,166482.154762,...,504.460329,0.654762,0.0,0.023810,0.345238,0.0,0.0,0.0,7.704307e+06,3.369959e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96.6.49.94,{443},"{172.31.64.69, 172.31.65.65, 172.31.65.113}","{51987, 49972, 50030}",{6},{Benign},1.890667e+03,3.000000,1.000000,31.000000,0.000000,...,6.200000,0.000000,0.0,0.000000,1.000000,0.0,0.0,0.0,0.000000e+00,0.000000e+00
96.6.55.102,{443},{172.31.69.4},{49802},{6},{Benign},1.000000e+00,2.000000,0.000000,31.000000,0.000000,...,20.666667,1.000000,0.0,0.000000,1.000000,0.0,0.0,0.0,0.000000e+00,0.000000e+00
97.68.80.146,{37767},{172.31.65.76},{445},{6},{Benign},4.814400e+04,3.000000,1.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,1.000000,0.000000,0.0,0.0,0.0,0.000000e+00,0.000000e+00


В связи с тем, что сгрупированы только данные внутри блоков и оставлены только уникальные IP-адреса источников даннх только внутри каждого болка (chunk) из файла - тем не менее между блоками IP-адреса могут пересекаться. Необходимо провести аналогичную группировку данных по всей собранной таблице, попутно вычислив средние показатели для непрерывных переменных , и собрать получившиеся множества категориальных переменных в единый блок для каждого ip-адреса.

Т.е. из каждого блока (chunk) для конкретного ip-адреса собрать вместе все ip-адреса назначения, протоколы, порты источника, порты назначения и т.п. Но сделать это простым методом не удастся. Т.к. в каждом блоке (chunk) данные были собраны во множества - нет возмоности средствами pandas объединить множества при очередной агрегации. Поэтому для большего удобства объединяем множества в списки (получается список множеств) и позже его обрабатываем. 

In [12]:
# вновь агрегируем данные по ip-адресу источника уже по единой таблице, вычисляя средние значения числовых параметров
cnt_data = intermediate_res.groupby(['Src IP']).mean()[['Flow Duration', 'Tot Fwd Pkts', 'Tot Bwd Pkts', 'TotLen Fwd Pkts', 
                                           'TotLen Bwd Pkts', 'Fwd IAT Mean' , 'Bwd IAT Mean' , 'Fwd PSH Flags', 
                                           'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'Pkt Len Mean', 
                                           'SYN Flag Cnt', 'RST Flag Cnt', 'PSH Flag Cnt', 'ACK Flag Cnt' , 
                                           'URG Flag Cnt', 'CWE Flag Count', 'ECE Flag Cnt', 'Idle Mean', 'Active Mean']]

# вновь агрегируем данные по ip-адресу источника уже по единой таблице, объединяя множества категориальных переменных
# в списки множеств (как описано выше)
dst_ip_data = intermediate_res.groupby(['Src IP'])['Dst IP'].apply(np.hstack).to_frame()
protocol_data = intermediate_res.groupby(['Src IP'])['Protocol'].apply(np.hstack).to_frame()
dst_port_data = intermediate_res.groupby(['Src IP'])['Dst Port'].apply(np.hstack).to_frame()
src_ports_data = intermediate_res.groupby(['Src IP'])['Src Port'].apply(np.hstack).to_frame()
labels_data = intermediate_res.groupby(['Src IP'])['Label'].apply(np.hstack).to_frame()

# собираем отдельные блоки данных в единую таблицу
result_data = dst_ip_data.join(protocol_data).join(dst_port_data).join(src_ports_data).join(labels_data).join(cnt_data)
result_data

,Dst IP,Protocol,Dst Port,Src Port,Label,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,...,Pkt Len Mean,SYN Flag Cnt,RST Flag Cnt,PSH Flag Cnt,ACK Flag Cnt,URG Flag Cnt,CWE Flag Count,ECE Flag Cnt,Idle Mean,Active Mean
Src IP,,,,,,,,,,,,,,,,,,,,,
1.0.132.203,[{172.31.64.91}],[{17}],[{38840}],[{60449}],[{Benign}],2.002600e+04,2.000000,0.000000,393.000000,0.000000,...,141.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1.0.176.98,"[{172.31.67.104}, {172.31.67.48}, {172.31.64.4...","[{6}, {6}, {6}, {6}, {6}, {6}, {6}, {6}, {6}, ...","[{445}, {445}, {445}, {445}, {445}, {445}, {44...","[{15281}, {50483}, {52319}, {30317}, {48026}, ...","[{Benign}, {Benign}, {Benign}, {Benign}, {Beni...",6.889161e+05,4.602273,3.079545,225.170455,177.477273,...,35.947596,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1.1.196.153,"[{172.31.64.99}, {172.31.65.48}, {172.31.65.48...","[{6}, {6}, {6}, {6}]","[{445}, {445}, {445}, {445}]","[{63015}, {56757}, {58357}, {63057}]","[{Benign}, {Benign}, {Benign}, {Benign}]",6.709760e+05,4.500000,2.750000,184.250000,188.500000,...,31.817308,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1.1.207.23,"[{172.31.67.19}, {172.31.67.19}, {172.31.64.14...","[{6}, {6}, {6}, {6}, {6}, {6}, {6}]","[{445}, {445}, {445}, {445}, {445}, {445}, {445}]","[{63898}, {63869}, {56816}, {56785}, {54095}, ...","[{Benign}, {Benign}, {Benign}, {Benign}, {Beni...",5.310179e+05,3.857143,2.285714,135.000000,73.142857,...,20.814286,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1.1.223.186,"[{172.31.67.62}, {172.31.67.62}, {172.31.67.62}]","[{6}, {6}, {6}]","[{445}, {445}, {445}]","[{26367}, {26340}, {26290}]","[{Benign}, {Benign}, {Benign}]",6.478580e+05,4.333333,3.000000,219.666667,114.000000,...,33.366667,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99.229.225.189,"[{172.31.65.25}, {172.31.65.25}]","[{6}, {6}]","[{30303}, {30303}]","[{54058}, {52240}]","[{Benign}, {Benign}]",2.999468e+06,2.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
99.230.229.253,[{172.31.64.109}],[{6}],[{30303}],[{64634}],[{Benign}],2.160461e+06,3.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
99.26.255.249,[{172.31.64.113}],[{6}],[{3389}],[{54288}],[{Benign}],7.665771e+07,8.000000,17.000000,1042.000000,1766.000000,...,108.000000,0.0,0.0,1.0,0.0,0.0,0.0,0.0,19046115.0,19519369.0


На этом этапе будем обрабатывать получившиеся списки множеств категориальных переменных. С такой структурой данныхработать сложно, поэтому необходимо ее упрощать.

В настоящий момент каждая строка result_data - это строка с данными для одного ip-адреса источника: список множеств ip-адресов комьютеров, на которые отправлялись сетевые пакеты, список множеств портов с которых отправлялись пакеты, список множеств портов на которые отправлялись пакеты, среднее значение интервала обмена данными, среднее значение длины пакета, среднее значение количества пакетов, отправлемых с данного комьютера во время одного сеанса и т.д.


Что из себя представляет структура "список множеств". Например, для строки 5 таблицы result_data значение её поля Dst IP
(ip-адрес, куда с источника отправлялись данные) имеет следующий вид:
[{172.31.67.62}, {172.31.67.62}, {172.31.67.62}]

Это список из 3х множеств в каждом из которых по одному элементу. При этом - элементы во множествах одинаковые. Логично
будет объединить все множества категориальных переменных, оставив только уникальные значения. В результате получим
следующий список ip-адресов, на которые отправлялись данные с источника: [172.31.67.62]

Единственная проблема - в датафрейм pandas нельза записывать множества. Поэтому мы сделаем из элементов множеств - строки
и объединим их через пробел. Так для поля Dst IP получим строку из ip-адресов, разделенных пробелами, для поля Dst Port - 
строку из номеров портов, разделенных пробелами и т.д. 

In [13]:
# перебираем все строки общей таблицы
for index, row in result_data.iterrows():
    #формируем пустое множество
    p = set()
    # берем поле категориальной переменной Dst IP. Это список. Получаем каждое множество из этого списка
    for d in row['Dst IP']:
        # и объединяем со множеством p
        p |= d
    # формируем из элементов итогового множества - строки и объединяем их через пробелы
    # результат записываем в нашу таблицу в исходное поле, откуда брали данные для преобразования
    result_data.loc[index,'Dst IP'] = " ".join(map(str, p))
    
    # выполняем описанные выше операции для всех категориальных переменных
    p = set()
    for d in row['Protocol']:
        p |= d
    result_data.loc[index,'Protocol'] = " ".join(map(str, p))
    
    p = set()
    for d in row['Dst Port']:
        p |= d
    result_data.loc[index,'Dst Port'] = " ".join(map(str, p))
    
    p = set()
    for d in row['Src Port']:
        p |= d
    result_data.loc[index,'Src Port'] = " ".join(map(str, p))
    
    p = set()
    for d in row['Label']:
        p |= d
    result_data.loc[index,'Label'] = " ".join(map(str, p))

# Теперь можно получить для любого ip-адреса источника значение поля, например, Dst IP, 
#разбить хранящуюся там строку по пробелам и получить список IP-адресов назначения, 
# на которые отправлялись пакеты с  этого источника. И так для любой категориальной переменной
result_data

,Dst IP,Protocol,Dst Port,Src Port,Label,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,...,Pkt Len Mean,SYN Flag Cnt,RST Flag Cnt,PSH Flag Cnt,ACK Flag Cnt,URG Flag Cnt,CWE Flag Count,ECE Flag Cnt,Idle Mean,Active Mean
Src IP,,,,,,,,,,,,,,,,,,,,,
1.0.132.203,172.31.64.91,17,38840,60449,Benign,2.002600e+04,2.000000,0.000000,393.000000,0.000000,...,141.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1.0.176.98,172.31.69.8 172.31.67.38 172.31.66.74 172.31.6...,6,445,52097 24706 31749 23054 19727 18706 13588 5838...,Benign,6.889161e+05,4.602273,3.079545,225.170455,177.477273,...,35.947596,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1.1.196.153,172.31.65.48 172.31.64.99,6,445,63057 58357 56757 63015,Benign,6.709760e+05,4.500000,2.750000,184.250000,188.500000,...,31.817308,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1.1.207.23,172.31.64.14 172.31.67.19,6,445,56816 56785 64357 57734 63898 63869 54095,Benign,5.310179e+05,3.857143,2.285714,135.000000,73.142857,...,20.814286,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1.1.223.186,172.31.67.62,6,445,26290 26340 26367,Benign,6.478580e+05,4.333333,3.000000,219.666667,114.000000,...,33.366667,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99.229.225.189,172.31.65.25,6,30303,52240 54058,Benign,2.999468e+06,2.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
99.230.229.253,172.31.64.109,6,30303,64634,Benign,2.160461e+06,3.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
99.26.255.249,172.31.64.113,6,3389,54288,Benign,7.665771e+07,8.000000,17.000000,1042.000000,1766.000000,...,108.000000,0.0,0.0,1.0,0.0,0.0,0.0,0.0,19046115.0,19519369.0


In [16]:
# отфильтруем строки таблицы по метке (ddos - источник отправлял пакеты для ddos атаки)
ddos_data = result_data[result_data['Label'] == 'ddos']
# запишем отфильтрованную таблицу в отдельный файлd dos_src.csv
ddos_data.to_csv("ddos_src.csv")
# IP-адресов в этой таблице немного, однако главное что мы понимаем, для
# каких комьютеров генерировать данные ddos-атаки. И знаем все показатели пакетов атаки
len(ddos_data)

34

In [17]:
# отфильтруем строки таблицы по метке (Benign - источник отправлял пакеты для обычных пользвательских запросов)
benign_data = result_data[result_data['Label'] == 'Benign']
# запишем и эти данные в csv файл. Он значительно больше. Это связано с тем, что нормального трафика в сети намного,
#больше чем аномального в этом подмножестве таблицы оказалось 36724 адреса. 
#Можем использовать эти данные для генерации пакетов чистых пользовательских запросов
len(benign_data)

36724